In [9]:
!pip install pandas yfinance matplotlib numpy

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Função para calcular o beta
def calc_beta(retornos_ativo, retornos_mercado):
    """
    Calcula o beta de um ativo em relação ao mercado.

    :param retornos_ativo: Série ou array dos retornos do ativo.
    :param retornos_mercado: Série ou array dos retornos do mercado.
    :return: Beta do ativo (valor escalar).
    """
    if len(retornos_ativo) == 0 or len(retornos_mercado) == 0:
        raise ValueError("Dados insuficientes para calcular o beta.")
    cov = np.cov(retornos_ativo, retornos_mercado)[0, 1]  # Covariância entre ativo e mercado
    var_mercado = np.var(retornos_mercado)  # Variância do mercado
    if var_mercado == 0:
        raise ValueError("A variância do mercado é zero, impossibilitando o cálculo do beta.")
    beta = cov / var_mercado
    return float(beta)  # Garantir que o retorno seja um valor escalar

# Função para obter dados históricos
def show_data(ticker, periodo, intervalo):
    """
    Obtém os dados históricos de um ativo utilizando o yfinance.

    :param ticker: Ticker do ativo.
    :param period: Período dos dados (padrão: últimos 6 meses).
    :param interval: Intervalo de tempo dos dados (padrão: 1 dia).
    :return: DataFrame com os preços ajustados de fechamento do ativo.
    """
    try:
        dados = yf.download(ticker, period=periodo, interval=intervalo)
        if dados.empty:
            raise ValueError(f"Nenhum dado encontrado para o ticker {ticker}. Verifique o ticker ou o período solicitado.")
        return dados[['Close']]  # Retorna apenas a coluna de preços de fechamento ajustados
    except Exception as e:
        raise ValueError(f"Erro ao obter dados para o ticker {ticker}: {e}")

# Função para exibir informações de debug
def print_debug(precos_ativo, precos_mercado, retornos_ativo, retornos_mercado):
    print("\n--- Debug ---")
    print("Preços do Ativo:")
    print(precos_ativo.head())  # Mostrar as primeiras 5 linhas dos preços do ativo
    print("\nPreços do Mercado:")
    print(precos_mercado.head())  # Mostrar as primeiras 5 linhas dos preços do mercado
    print("\nRetornos do Ativo:")
    print(retornos_ativo.head())  # Mostrar os primeiros retornos do ativo
    print("\nRetornos do Mercado:")
    print(retornos_mercado.head())  # Mostrar os primeiros retornos do mercado

# Função principal
def main():
    # Entradas do usuário
    ativo_ticker = input("Digite o ticker do ativo (exemplo: PETR4): ").strip().upper()

    # Ajuste para ativos brasileiros
    if ativo_ticker and not ativo_ticker.endswith(".SA"):
        ativo_ticker = ativo_ticker + ".SA"

    mercado_ticker = input("Digite o ticker do mercado de referência (deixe vazio para usar iBOVESPA): ").strip()
    if not mercado_ticker:
        mercado_ticker = "^BVSP"

    try:
        # Obter dados históricos para os últimos 6 meses
        precos_ativo = show_data(ativo_ticker, "1y", "1wk")
        precos_mercado = show_data(mercado_ticker, "1y", "1wk")

        # Verificar se os dados foram baixados corretamente
        if precos_ativo.empty or precos_mercado.empty:
            raise ValueError("Dados do ativo ou mercado não estão disponíveis.")

        # Alinhar as datas para ambos os ativos
        precos_ativo, precos_mercado = precos_ativo.align(precos_mercado, join='inner')

        # Calculando retornos logarítmicos
        retornos_ativo = np.log(precos_ativo / precos_ativo.shift(1)).dropna()
        retornos_mercado = np.log(precos_mercado / precos_mercado.shift(1)).dropna()

        # Chamar a função de debug para exibir os dados intermediários
        print_debug(precos_ativo, precos_mercado, retornos_ativo, retornos_mercado)

        # Verificar se há dados suficientes para o cálculo
        if len(retornos_ativo) == 0 or len(retornos_mercado) == 0:
            raise ValueError("Dados insuficientes para cálculo dos retornos.")

        # Calculando o beta
        beta = calc_beta(retornos_ativo, retornos_mercado)

        # Exibindo resultados
        print(f"Beta calculado para o ativo {ativo_ticker} em relação ao mercado {mercado_ticker}: {beta:.4f}")

        # Visualização dos preços
        plt.figure(figsize=(10, 6))
        plt.plot(precos_ativo.index, precos_ativo['Close'], label=f'{ativo_ticker}')
        plt.plot(precos_mercado.index, precos_mercado['Close'], label=f'{mercado_ticker}', linestyle='--')
        plt.title('Preços históricos')
        plt.xlabel('Data')
        plt.ylabel('Preço ajustado')
        plt.legend()
        plt.show()

    except ValueError as e:
        print(f"Erro: {e}")

# Chamar a função principal
if __name__ == "__main__":
    main()


Digite o ticker do ativo (exemplo: PETR4): petr4
Digite o ticker do mercado de referência (deixe vazio para usar iBOVESPA): 


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


--- Debug ---
Preços do Ativo:
Empty DataFrame
Columns: []
Index: [2024-01-29 00:00:00, 2024-02-05 00:00:00, 2024-02-12 00:00:00, 2024-02-19 00:00:00, 2024-02-26 00:00:00]

Preços do Mercado:
Empty DataFrame
Columns: []
Index: [2024-01-29 00:00:00, 2024-02-05 00:00:00, 2024-02-12 00:00:00, 2024-02-19 00:00:00, 2024-02-26 00:00:00]

Retornos do Ativo:
Empty DataFrame
Columns: []
Index: [2024-01-29 00:00:00, 2024-02-05 00:00:00, 2024-02-12 00:00:00, 2024-02-19 00:00:00, 2024-02-26 00:00:00]

Retornos do Mercado:
Empty DataFrame
Columns: []
Index: [2024-01-29 00:00:00, 2024-02-05 00:00:00, 2024-02-12 00:00:00, 2024-02-19 00:00:00, 2024-02-26 00:00:00]
Erro: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().



/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
<ipython-input-29-d6f7b64e1ca7>:17: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov = np.cov(retornos_ativo, retornos_mercado)[0, 1]  # Covariância entre ativo e mercado
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scala